<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# Digitizer Internal Trigger

This script uses the Digitizer in internal Trigger mode. This is the basic Trigger mode<br>

The Signal generator is confige:<br>
DAC frequency: 2GHz;<br>
Channel 1:<br>
 - Sinuse wave form;
 - amplitude 0.5V;
 
Channel 2: 
 - Triangular wave form;
 - amplitude 0.5V;
          
CH1 should be connected to Digitizer channel 1.<br> 
CH2 should be connected to Digitizer channel 2.<br>

The Digitizer is confige:
 - Mode           : DUAL 
 - Sample Rate    : 2.7GHz
 - Trigger source : CPU (internal)
 - The Digitizer External Trigger threshold is set to 0.7V

This script uses 4 Frames with 4800 samples each.<br>

Start the Digitizer;<br>
 - send 4 internal Triggers.<br>
 - Check that the MULTIY_FRAME_DONE bit goes 'high'.<br>
 
Stop the Digitizer;<br>

Read the DDR memory and polt the signals.<br>

check the proteus registers and close the script.<br>


In [ ]:
import os
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst

In [ ]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

In [ ]:
admin = TepAdmin()
    
# Get list of available PXI slots
slot_ids = admin.get_slot_ids()

# Assume that at least one slot was found
sid = slot_ids[0]

 # Open a single-slot instrument:
inst = admin.open_instrument(slot_id=sid)

In [ ]:
# Set the default paranoia-level (0, 1 or 2)
inst.default_paranoia_level = 2 # good for debugging

In [ ]:
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

# Get the model name
resp = inst.send_scpi_query(":SYST:iNF:MODel?")
print("Model: " + resp)

# Infer the natural DAC waveform format
if 'P9082' in resp:
    dac_mode = 8
else:
    dac_mode = 16
print("DAC waveform format: {0} bits-per-point".format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':SOUR:FREQ 2.0e9')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

## Configure Generator

In [ ]:
# Build two AWG segment that will be used as innput for the digitizer
cycleLen = 1024
numCycles = 1
segLen = cycleLen * numCycles

cycleLen = 1024
numCycles = 8
segLen = cycleLen * numCycles

# Build waveform (sine wave)
x = np.linspace(
    start=0, stop=2 * np.pi * numCycles, num=segLen, endpoint=False)

y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
y = y.astype(data_type)

# download it to segment 2 of channel 1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEF 1,' + str(segLen))
inst.send_scpi_cmd(':TRAC:SEL 1')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 1')
inst.send_scpi_cmd(':SOUR:VOLT 0.5')
inst.send_scpi_cmd(':OUTP ON')
# download the waveform to the selected segment
inst.write_binary_data(':TRAC:DATA', y)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

y = np.sin(x)
y = np.arcsin(y)* 2 * half_dac / np.pi + half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
y = y.astype(data_type)

# download it to segment 2 of channel 1
inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':TRAC:DEF 2,' + str(segLen))
inst.send_scpi_cmd(':TRAC:SEL 2')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 2')
inst.send_scpi_cmd(':SOUR:VOLT 0.5')
inst.send_scpi_cmd(':OUTP ON')
# download the waveform to the selected segment
inst.write_binary_data(':TRAC:DATA', y)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

## Setup Digitizer

In [ ]:
# Setup the digitizer in two-channels mode
inst.send_scpi_cmd(':DIG:MODE DUAL')
inst.send_scpi_cmd(':DIG:FREQ 2700MHZ')

# Allocate four frames of 4800 samples
numframes, framelen = 4, 4800
cmd = ':DIG:ACQuire:FRAM:DEF {0},{1}'.format(numframes, framelen)
inst.send_scpi_cmd(cmd)

# Select the frames for the capturing 
# (all the four frames in this example)
capture_first, capture_count = 1, numframes
cmd = ":DIG:ACQuire:FRAM:CAPT {0},{1}".format(capture_first, capture_count)
inst.send_scpi_cmd(cmd)

# Enable capturing data from channel 1
inst.send_scpi_cmd(':DIG:CHAN:SEL 1')
inst.send_scpi_cmd(':DIG:CHAN:STATE ENAB')
# Select the internal-trigger as start-capturing trigger:
inst.send_scpi_cmd(':DIG:TRIG:SOURCE CPU')

# Enable capturing data from channel 2
inst.send_scpi_cmd(':DIG:CHAN:SEL 2')
inst.send_scpi_cmd(':DIG:CHAN:STATE ENAB')
# Select the internal-trigger as start-capturing trigger:
inst.send_scpi_cmd(':DIG:TRIG:SOURCE CPU')


# Clean memory 
inst.send_scpi_cmd(':DIG:ACQ:ZERO:ALL')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
print("Set Digitizer: DUAL mode; internal Trigger")


#### Acquire signals into Memory

In [ ]:
# Stop the digitizer's capturing machine (to be on the safe side)
inst.send_scpi_cmd(':DIG:INIT OFF')

# Start the digitizer's capturing machine
inst.send_scpi_cmd(':DIG:INIT ON')
# Generate 4 software-triggers
for _ in range(capture_count):
    inst.send_scpi_cmd(':DIG:TRIG:IMM')
    time.sleep(0.1) # more than  enough for capturing single frame
    # Query the status
    resp = inst.send_scpi_query(":DIG:ACQuire:FRAM:STATus?")
    print(resp)

print("Capture Done")

# Stop the digitizer's capturing machine (to be on the safe side)
inst.send_scpi_cmd(':DIG:INIT OFF')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
print("Digitizer: Finish Acquisition")

In [ ]:
# start pulse counter
inst.send_scpi_cmd(':SHELL:REG 0x00406060,0x00000001')
inst.send_scpi_cmd(':SHELL:REG 0x00406060,0x00000000')
# wait for 100mSec
done = 0
while done == 0:
    resp = inst.send_scpi_query(':SHELL:REG? 0x00406020')
    a = np.bitwise_and(int(resp, 16), 0x0004)
    if a == 0:
        done = 1 
Ch0 = inst.send_scpi_query(':SHELL:REG? 0x00406068')
Ch2 = inst.send_scpi_query(':SHELL:REG? 0x0040606c')


## Read all frames from Memory

In [ ]:
# Choose which frames to read (all in this example)
inst.send_scpi_cmd(':DIG:DATA:SEL ALL')

# Choose what to read 
# (only the frame-data without the header in this example)
inst.send_scpi_cmd(':DIG:DATA:TYPE FRAM')

# Get the total data size (in bytes)
resp = inst.send_scpi_query(':DIG:DATA:SIZE?')
num_bytes = np.uint32(resp)
print('Total size in bytes: ' + resp)
print()

# Read the data that was captured by channel 1:
inst.send_scpi_cmd(':DIG:CHAN:SEL 1')

wavlen = num_bytes // 2

wav1 = np.zeros(wavlen, dtype=np.uint16)

rc = inst.read_binary_data(':DIG:DATA:READ?', wav1, num_bytes)

# Read the data that was captured by channel 2:
inst.send_scpi_cmd(':DIG:CHAN:SEL 2')

wavlen = num_bytes // 2

wav2 = np.zeros(wavlen, dtype=np.uint16)

rc = inst.read_binary_data(':DIG:DATA:READ?', wav2, num_bytes)

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
print("read data from DDR")


In [ ]:
# RMS calculation
rmsCh0 = np.zeros(numframes)
rmsCh2 = np.zeros(numframes)
for iframe in range (numframes):
    framewav = wav1[iframe*framelen : (iframe+1)*framelen]
    rmsCh0[iframe] = np.sqrt(np.mean(np.square(1000*(framewav/4096))))
    framewav = wav2[iframe*framelen : (iframe+1)*framelen]
    rmsCh2[iframe] = np.sqrt(np.mean(np.square(1000*(framewav/4096))))


## Plot Signals

In [ ]:
# Plot the samples
totlen = numframes * framelen
x = range(numframes * framelen)
plt.figure(1)
ax1 = plt.subplot(211)
ax1.set_xticks(np.arange(0, totlen, framelen))
ax1.grid(True)
ax1.plot(x, wav1)
ax1.set_ylabel('CH1')

ax2 = plt.subplot(212, sharex=ax1)
ax2.set_xticks(np.arange(0, totlen, framelen))
ax2.grid(True)
ax2.plot(x, wav2)
ax2.set_ylabel('CH2')

plt.show()

print("freq Channel 1: {0} Hz".format(int(Ch0,16)*10))
print("freq Channel 2: {0} Hz".format(int(Ch2,16)*10))
print ("")
print ("RMS Channel 0")
print ("frame0 {0}mV".format(int(rmsCh0[0])))
print ("frame1 {0}mV".format(int(rmsCh0[1])))
print ("frame2 {0}mV".format(int(rmsCh0[2])))
print ("frame2 {0}mV".format(int(rmsCh0[3])))
print ("")
print ("RMS Channel 2")
print ("frame0 {0}mV".format(int(rmsCh2[0])))
print ("frame1 {0}mV".format(int(rmsCh2[1])))
print ("frame2 {0}mV".format(int(rmsCh2[2])))
print ("frame2 {0}mV".format(int(rmsCh2[3])))


## End

In [ ]:
# close connection
inst.close_instrument()
admin.close_inst_admin()